In [56]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import plotly.express as px #if using plotly
import folium
import warnings


## Research Questions

### 2) Is there a significant difference in the proportion of minorities within a given flaring buffer zone versus outside the buffer? 

#### Ultimate goal is a comparison of six different buffers (of 100; 400; 800; 1,000; 1,600; and 2,000 m) following Czolowski et al, 2017)

Null: Proportion of BG in a buffer is independent of minority status.
Alt:  Proportion of BG in a buffer is related to minority status.
  
Test stat: Difference in weighted minority proportions.

Weighted minority proportion = sumproduct(minority_prop * intersect_prop) / sum(intersect_prop)
minority_prop is the proportion of minorities in a block group
intersect_prop is the proportion of BG area residing in the buffer zone

Permutation testing: Permute the intersect_prop variable, holding the fraction of minorities in each BG constant.
 
Process   
a) Calculate the actual minority proportions of the aggregrate in-buffer and out-buffer areas
Using 2km buffer for starters (Czolowski, 2017)
b) Scramble the intersect_prop variable, holding the minority_prop variable constant. 
c) Calculate number of simulated proportions that match or exceed the actual proportion. Calculate p-value.


## Setup / load data

In [57]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

In [58]:
ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/tl_2022_06_bg.shp")  # CA block groups

In [59]:
# Load census block group level data from EJscreen
# Source: https://www.epa.gov/ejscreen/download-ejscreen-data
# This is the 2017-2021 5-year ACS average data
ejscreen = pd.read_excel("data/CA_EJSCREEN_2022_Full_with_AS_CNMI_GU_VI.xlsx", index_col=None, header=0)

In [60]:
# read in cleaned and combined flares data
all_flares = gp.read_file("data/all_flares.shp")

In [61]:
# set common crs for project
#projcrs = 4326

# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 
ca_state = ca_state.to_crs(meters_crs)
ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)
all_flares = all_flares.to_crs(meters_crs)


ca_bg.rename(columns={'GEOID':'ID'}, inplace=True)  # match column names for merging
ca_bg['ID'] = ca_bg['ID'].astype(np.int64)

In [62]:
print(f"{len(ejscreen['ID'])} block groups in the EJScreen data \n")
print(f"{len(ca_bg['ID'])} block groups in the CA block group shapefile\n")
ca_bg_joined = pd.merge(ca_bg, ejscreen, on='ID')
print(f"{len(ca_bg['ID'].unique())-len(ca_bg_joined)} block groups are missing after merge")


25607 block groups in the EJScreen data 

25607 block groups in the CA block group shapefile

0 block groups are missing after merge


### Discuss w RMI: dropping the three aquatic buffers

In [157]:
# There are three BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# IDs: 60839900000, 61119901000, 60379902000

# ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['ID'].isin(ids_to_drop)]

In [159]:
# subset flares to only those in Cali
ca_flares = gp.sjoin(all_flares, ca_counties, how = "inner", predicate = 'within')
print(f'Flares found: {len(ca_flares)}')
ca_flares.drop('index_right', axis=1, inplace=True)

# set col list for BCM_avg calculation
col_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[col_list].mean(axis=1)  

Flares found: 117


In [64]:


# ca_polys_subset = ca_polys_subset.to_crs(meters_crs)
# ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

In [65]:
# # Create "flares present" binary dependent variable
# ca_polys_subset = gp.sjoin(ca_bg_joined, ca_flares, how='inner', predicate='contains')
# ca_bg_joined = ca_bg_joined.assign(flares_present=ca_bg_joined['geometry'].isin(ca_polys_subset['geometry']).astype(int))

# print(f"Block groups containing flares (0=no, 1=yes):\n")
# print(ca_bg_joined['flares_present'].value_counts())

## 2km Buffer Analysis

1) set buffers around flares  
2) Subset census data to only to race and age columns  
3) count # of people and create proportion of those columns that are anywhere within the combined buffer  
4) same for outside the buffer  
5) Create table that summarizes the proportions for in-buffer and out-buffer

In [160]:
# 1) set 2km buffers around flares and unary_union() into single multipolygon
#ca_flares = ca_flares.to_crs(meters_crs)
ca_flares["buffer_2000m"] = ca_flares['geometry'].buffer(distance = 2000)

In [161]:
# ID, ACSTOTPOP, UNDER5, OVER64, MINORPOP, LOWINCOME, D_PM25_2
# total population, under 5yr, over 64yr, people of color, low income, EJ index for PM2.5

bg_formodel = ca_bg_joined[['ID', 'CNTY_NAME', 'Shape_Area', 'ACSTOTPOP', 'UNDER5', 'OVER64', 'MINORPOP', 'LOWINCOME', 'D_PM25_2', 'geometry']]

In [162]:
warnings.filterwarnings('ignore')
bg_formodel.loc[:, 'shape_area_new'] = bg_formodel.geometry.area

In [163]:
# save to csv for use with initial permutation testing
# bg_formodel.to_csv("data/bgformodel_permutation.csv")

In [176]:
# total BG in California
len(bg_formodel)

25604

In [177]:
# total BG that intersect with any buffer
len(bg_inbuffer)

238

In [166]:
# sjoin() doesn't seem to allow picking a specific geometry col.
# Have to manually set it to the buffers rather than the flare points
ca_flares = ca_flares.set_geometry('buffer_2000m')  # set to the buffers rather than the points

# create unary_union of flares, then change back to geodf for processing
# crs must be geographic not projected to do a unary union
temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs)  # convert back to geodf for processing

In [181]:
# Spatial overlay operation to find only the areas that are in both geometries. 
# i.e. only the block group areas that are within any buffer zone

# this can help with "bad" geometries. looks at both inputs and repairs any bad geometries. 
# make_valid=True
bg_inbuffer = gp.overlay(bg_formodel, all_buffers, how='intersection')  # could look at keep_geom=False

# Create new 'area' column for the areas of the intersections
#bg_inbuffer = bg_inbuffer.to_crs(meters_crs)
bg_inbuffer['intersect_area'] = bg_inbuffer.area
# Calculate the proportion of each block group within the buffer zone
bg_inbuffer['intersect_prop'] = bg_inbuffer['intersect_area'] / bg_inbuffer['shape_area_new']

In [182]:
# Rename geometry col to intersect_geom so it's clear these geoms are just the intersections
bg_inbuffer.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging

# merge with ca_bg block groups to get full BG polygon geoms back in the df
bg_inbuffer = pd.merge(bg_inbuffer, ca_bg, on='ID')

# Rename geometry column for clarity
bg_inbuffer.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging


In [220]:
# ensure geometry column is set to the full BG polygons, not intersections
bg_inbuffer = bg_inbuffer.set_geometry('bg_geom')  # set to the buffers rather than the points

# merge the intersected data back into the full df of all block groups
test = gp.sjoin(bg_formodel, bg_inbuffer, how = "left", predicate = 'contains')

In [221]:
# Length of non-NA intersection proportions should match
# the length of intersections found in the bg_inbuffer overlay output

assert test['intersect_prop'].count() == len(bg_inbuffer), "The count of non-null values in the 'intersect_prop' column does not match the length of the original overlay"

In [222]:
# Filter cols to just what's needed
test = test[['ID_left', 'CNTY_NAME_left', 'ACSTOTPOP_left', 'MINORPOP_left', 'D_PM25_2_left', 'shape_area_new_left', 'intersect_area', 'intersect_prop', 'geometry', 'intersect_geom']]

# clean things up after the sjoin
new_cols = [col.replace('_left', '') if col.endswith('_left') else col for col in test.columns]
test.columns = new_cols

In [213]:
# double check the merge result
test[test['intersect_geom'].notnull()].sample(50).explore()

In [237]:
# Rename geometry column for clarity
test.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

# Replace NAs in both the intersect_prop and intersect_area with 0 for BG that have no intersection
no_intersection = test['intersect_prop'].isna() | test['intersect_area'].isna()

# Update the values in the 'intersect_prop' and 'intersect_area' columns of those rows to 0 using the loc method
test.loc[no_intersection, ['intersect_prop', 'intersect_area']] = 0


In [245]:
test.sample(10)

,ID,CNTY_NAME,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,intersect_area,intersect_prop,bg_geom,intersect_geom
17393,60730126002,San Diego County,934,748,42.89,"250,552.17",0.00,0.00,"POLYGON ((273058.180 -594284.330, 273104.476 -...",None
23660,60133141051,Contra Costa County,1405,1258,63.33,"697,622.54","34,395.64",0.05,"POLYGON ((-172325.868 2543.653, -172304.862 25...","POLYGON ((-170580.894 2085.094, -170554.578 20..."
3831,60371954003,Los Angeles County,876,567,40.07,"244,360.08",0.00,0.00,"POLYGON ((158421.151 -434104.848, 158522.183 -...",None
12604,60552017006,Napa County,407,86,19.33,"10,034,624.61",0.00,0.00,"POLYGON ((-216263.526 59012.964, -216228.800 5...",None
18157,60170315032,El Dorado County,1564,243,12.84,"81,515,917.03",0.00,0.00,"POLYGON ((-82173.323 57261.319, -82166.662 572...",None
15314,60770014003,San Joaquin County,635,374,57.22,"328,611.17",0.00,0.00,"POLYGON ((-114242.180 -2376.206, -113945.064 -...",None
1348,60371352051,Los Angeles County,928,282,17.58,"1,732,273.16",0.00,0.00,"POLYGON ((122741.283 -424058.532, 122740.898 -...",None
1596,60379006101,Los Angeles County,1306,935,26.70,"2,639,474.69",0.00,0.00,"POLYGON ((174380.511 -364656.238, 174448.793 -...",None
9836,60530132001,Monterey County,2250,665,2.32,"45,131,492.33",0.00,0.00,"POLYGON ((-170960.744 -158700.231, -170955.124...",None
879,60377030032,Los Angeles County,1998,1208,32.12,"1,161,597.33",0.00,0.00,"POLYGON ((147970.639 -446233.819, 147973.675 -...",None


In [246]:
# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    test[var + '_intersect_count'] = test[var] * test['intersect_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    test[var + '_bg_totprop'] = test[var] / test['ACSTOTPOP']

In [319]:
len(test)

25604

### In/out buffer proportions

In [248]:
#prop_vars = ['ACSTOTPOP_bg_totprop', 'UNDER5_bg_totprop', 'OVER64_bg_totprop', 'MINORPOP_bg_totprop', 'LOWINCOME_bg_totprop']

in_buffer = np.sum(test['MINORPOP_bg_totprop'] * test['intersect_prop']) / np.sum(test['intersect_prop'])
out_buffer = np.sum(test['MINORPOP_bg_totprop'] * (1 - test['intersect_prop'])) / np.sum(1 - test['intersect_prop'])

# create a table from the in-buffer and outside-buffer proportions
# proportions = pd.concat([in_buffer, out_buffer], axis=1)
# proportions.columns = ['In_Buffer', 'Outside_Buffer']

# table that compares the in-buffer proportions to the outside-buffer proportions
temp = pd.DataFrame({'In_Buffer': [in_buffer], 'Outside_Buffer': [out_buffer]})
print(temp)

             In_Buffer       Outside_Buffer
0                 0.74                 0.60


### In/out buffer counts

In [249]:
# count_vars = ['ACSTOTPOP_intersect_count', 'UNDER5_intersect_count', 'OVER64_intersect_count', 'MINORPOP_intersect_count', 'LOWINCOME_intersect_count']

in_buffer_tot = test['MINORPOP_bg_totprop'].sum()
out_buffer_tot = test['ACSTOTPOP'].sum() - in_buffer_tot

# create a table from the in-buffer and outside-buffer counts
#counts = pd.concat([in_buffer_tot, out_buffer_tot], axis=1)
temp = pd.DataFrame({'In_Buffer': [in_buffer_tot], 'Outside_Buffer': [out_buffer_tot]})
# format w thousands separators
#counts[['In_Buffer_Count', 'Outside_Buffer_Count']] = counts[['In_Buffer_Count', 'Outside_Buffer_Count']].applymap('{:,.0f}'.format)

# table that compares the in-buffer counts to the outside-buffer counts
print(temp)

             In_Buffer       Outside_Buffer
0            15,392.49        39,330,630.51


#### Save to csv for permutation testing in other notebook

**Note** calcs are not correct yet.

In [318]:
# save to a csv for further processing in other notebooks
test.to_csv("data/bgformodel_permutation.csv")

## Visualization with Folium

In [309]:
# Convert the GeoDataFrame to the same CRS as the folium map (if necessary)

# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)

In [310]:
# # Choropleth of Block Groups

# folium.Choropleth(ca_bg_joined,
#                   data=ca_bg_joined,
#                   columns = ['ID', 'D_PM25_2'], 
#                   key_on='feature.properties.ID',
#                   fill_color="Reds",
#                   fill_opacity=0.7,
#                   line_opacity=0.2,
#                   legend_name="PM2.5 Index",
#                  name="Block Groups").add_to(m)


In [311]:
# Feature Group: All Block Groups

def style_function3(feature):
    return {
        'fillColor': 'grey',
        'color': 'grey',
        'fillOpacity': 0.05
    }

all_bg = folium.FeatureGroup(name='All BG', overlay=True)
folium.GeoJson(
    data=ca_bg["geometry"],
    style_function=style_function3
).add_to(all_bg)
all_bg.add_to(m)

In [312]:
# Feature Group: Buffers

def style_function1(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'fillOpacity': 0.2
    }

all_flares_buffers = folium.FeatureGroup(name='Flare Buffers', overlay=True)
folium.GeoJson(
    data=ca_flares["buffer_2000m"],
    style_function=style_function1
).add_to(all_flares_buffers)
all_flares_buffers.add_to(m)

In [313]:
# Feature Group: BG-Buffer Intersections

def style_function2(feature):
    return {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.1
    }

intersect = folium.FeatureGroup(name='BG-Flare Intersections', overlay=True)
folium.GeoJson(
    data=test['intersect_geom'],
    style_function=style_function2
).add_to(intersect)
intersect.add_to(m)

In [314]:
# # Add hover functionality
# test = test.to_crs("EPSG:3857")
# test = test.set_geometry('bg_geom')
# test_json = test.__geo_interface__

In [315]:
map_test = ca_bg_joined.sample(500)

In [316]:

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#999999', 
                                'color':'#999999', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = map_test,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ID','CNTY_NAME'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

In [320]:
# add hover functionality as child to map, add layering, display map
m.add_child(NIL)
m.keep_in_front(NIL)
folium.LayerControl().add_to(m)
m

In [86]:
# # Flares
# all_flares_points = folium.FeatureGroup(name='flare points', overlay=True)
# folium.GeoJson(data=ca_flares["geometry"]).add_to(all_flares_points)
# all_flares_points.add_to(m)